In [13]:
# library installations if necessary, make sure you're using .venv!

%pip install pandas
%pip install openpyxl
%pip install keyboard

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Imports

import pandas as pd
import re
import os
import keyboard

In [5]:
vt = pd.read_excel("ViolationTracker_21Aug2025_PHMSA_only.xlsx")
phmsa = pd.read_excel("PHMSA_RAW_DATA.xlsx")

# Helper function to make a numeric-only CPF key for PHMSA (e.g., "42025041NOA" -> "42025041")
def phmsa_cpf_key(s):
    if pd.isna(s):
        return ""
    m = re.match(r'\D*(\d+)', str(s))
    return m.group(1) if m else ""

phmsa["cpf_key"] = phmsa["CPF_Number"].apply(phmsa_cpf_key)

# Helper function that extracts CPF number from "info_source" column in GJF data.
# Try pattern 'cpf_123456789' first, else fallback to the longest digit run
def vt_cpf_from_info(url):
    if pd.isna(url):
        return ""
    txt = str(url)
    # This regex matches things like 'cpf_12345' or 'CPF-12345'
    m = re.search(r'(?i)cpf[_\-]?(\d{4,})', txt)   # case-insensitive, require >=4 digits
    if m:
        return m.group(1)
    # fallback: find all digit runs and return the longest (likely the CPF if present)
    runs = re.findall(r'(\d{4,})', txt)   # capture runs of 4+ digits
    if not runs:
        return ""
    # choose the longest run (if ties, first)
    runs_sorted = sorted(runs, key=lambda x: (-len(x), x))
    return runs_sorted[0]

vt["cpf_key_extracted"] = vt["info_source"].apply(vt_cpf_from_info)

# 3) Quick sanity counts
print("PHMSA distinct cpf_key count:", phmsa["cpf_key"].nunique())
print("GJF rows with extracted cpf_key:", (vt["cpf_key_extracted"] != "").sum())

# 4) Merge VT -> PHMSA on the cpf key
merged = vt.merge(phmsa, left_on="cpf_key_extracted", right_on="cpf_key", how="left", suffixes=("_vt", "_phmsa"))

# 5) Inspect mismatches
matched = merged[merged["CPF_Number"].notna()]
unmatched = merged[merged["CPF_Number"].isna()]
print("Matched rows:", len(matched))
print("Unmatched GJF rows after cpf merge:", len(unmatched))

# Save a sample of merged/unmatched for inspection
# merged.to_excel("VT_PHMSA_merged_by_cpf.xlsx", index=False)
# unmatched.to_excel("unmatched_rows.xlsx", index=False)


PHMSA distinct cpf_key count: 4944
GJF rows with extracted cpf_key: 718
Matched rows: 667
Unmatched GJF rows after cpf merge: 53


In [11]:
# Helper to normalize parent name
def normalize_parent(name):
    if pd.isna(name):
        return ""
    return re.sub(r'[^a-z0-9 ]', '', str(name).lower().strip())

# New columns for normalized names
matched["current_parent_name_n"] = matched["current_parent_name"].apply(normalize_parent)
matched["reporting_date_parent_n"] = matched["reporting_date_parent"].apply(normalize_parent)

# Create parent_changed flag
matched["parent_changed"] = matched["current_parent_name_n"] != matched["reporting_date_parent_n"]
print("Parent change flag counts:")
print(matched["parent_changed"].value_counts())

# Save full version
matched.to_excel("VT_PHMSA_matched_with_parentchange.xlsx", index=False)

# Filter only changed-parent entries
changed_only = matched[matched["parent_changed"] == True].copy()

# Save filtered version
changed_only.to_excel("VT_PHMSA_parentchanged_only.xlsx", index=False)

print(f"\nSaved {len(matched)} total rows to 'VT_PHMSA_matched_with_parentchange.xlsx'")
print(f"Saved {len(changed_only)} changed-parent rows to 'VT_PHMSA_parentchanged_only.xlsx'")


C:\Users\ashok\AppData\Local\Temp\ipykernel_12140\3028011393.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched["current_parent_name_n"] = matched["current_parent_name"].apply(normalize_parent)
C:\Users\ashok\AppData\Local\Temp\ipykernel_12140\3028011393.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched["reporting_date_parent_n"] = matched["reporting_date_parent"].apply(normalize_parent)
C:\Users\ashok\AppData\Local\Temp\ipykernel_12140\3028011393.py:12: SettingWithCopyWarning: 
A value i

Parent change flag counts:
parent_changed
False    475
True     192
Name: count, dtype: int64

Saved 667 total rows to 'VT_PHMSA_matched_with_parentchange.xlsx'
Saved 192 changed-parent rows to 'VT_PHMSA_parentchanged_only.xlsx'


In [ ]:
INPUT_FILE = "VT_PHMSA_parentchanged_only.xlsx"
OUTPUT_FILE = "VT_PHMSA_parentchanged_with_dates.xlsx"

df = pd.read_excel(INPUT_FILE)

# Ensure acquisition_exact_date column exists
if "acquisition_exact_date" not in df.columns:
    df["acquisition_exact_date"] = None

# Ensure acquisition_announcement_exact_date column exists
if "acquisition_announcement_exact_date" not in df.columns:
    df["acquisition_announcement_exact_date"] = None

acquisition_cache = {}
acquisition_announcement_cache = {}

valid_output_file = False
# Load cache from OUTPUT_FILE if available
if os.path.exists(OUTPUT_FILE):
    try:
        df_existing = pd.read_excel(OUTPUT_FILE)
        if "history_recap" in df_existing.columns:
            if "acquisition_exact_date" in df_existing.columns:
                existing_with_dates = df_existing.dropna(subset=["acquisition_exact_date"])
                acquisition_cache = dict(zip(existing_with_dates["history_recap"], existing_with_dates["acquisition_exact_date"]))
                print(f"Loaded {len(acquisition_cache)} cached acquisition dates from '{OUTPUT_FILE}'")
                # Merge back existing acquisition dates into df
                df = df.merge(
                    df_existing[["history_recap", "acquisition_exact_date"]],
                    on="history_recap",
                    how="left",
                    suffixes=("", "_existing")
                )
                df["acquisition_exact_date"] = df["acquisition_exact_date"].combine_first(df["acquisition_exact_date_existing"])
                df.drop(columns=["acquisition_exact_date_existing"], inplace=True)
            else:
                print(f"'{OUTPUT_FILE}' found but missing acquisition_exact_date column; starting with empty cache.")
            if "acquisition_announcement_exact_date" in df_existing.columns:
                existing_with_dates = df_existing.dropna(subset=["acquisition_announcement_exact_date"])
                acquisition_announcement_cache = dict(zip(existing_with_dates["history_recap"], existing_with_dates["acquisition_announcement_exact_date"]))
                print(f"Loaded {len(acquisition_announcement_cache)} cached acquisition announcement dates from '{OUTPUT_FILE}'")
                # Merge back existing acquisition dates into df
                df = df.merge(
                    df_existing[["history_recap", "acquisition_announcement_exact_date"]],
                    on="history_recap",
                    how="left",
                    suffixes=("", "_existing")
                )
                df["acquisition_announcement_exact_date"] = df["acquisition_announcement_exact_date"].combine_first(df["acquisition_announcement_exact_date_existing"])
                df.drop(columns=["acquisition_announcement_exact_date"], inplace=True)
            else:
                print(f"'{OUTPUT_FILE}' found but missing acquisition_announcement_exact_date column; starting with empty cache.")
        else:
            print(f"WARNING: '{OUTPUT_FILE}' found but missing history_recap; starting with empty cache.")
        valid_output_file = True
    except Exception as e:
        print(f"Error reading '{OUTPUT_FILE}': {e}")
else:
    print("No existing output file found, quitting.")

if valid_output_file:
    # Interactive acquisition date input
    print("\nPress ESC at any time to stop and save progress.\n")

    for idx, row in df.iterrows():
        # Check for keyboard escape before each iteration
        if keyboard.is_pressed("esc"):
            print("\nEscape pressed — saving progress...")
            break
        
        old_parent = str(row["reporting_date_parent_n"])
        new_parent = str(row["current_parent_name_n"])
        company_name = str(row["company"])
        recap = str(row["history_recap"])
        current_value = row.get("acquisition_exact_date", None)
        current_announcement_value = row.get("acquisition_announcement_exact_date", None)

        # Skip rows that already have a cached or filled date
        if recap in acquisition_cache and recap in acquisition_announcement_cache:
            df.at[idx, "acquisition_exact_date"] = acquisition_cache[recap]
            df.at[idx, "acquisition_announcement_exact_date"] = acquisition_announcement_cache[recap]
            continue
        if pd.notna(current_value) and pd.notna(current_announcement_value):
            acquisition_cache[recap] = current_value
            acquisition_announcement_cache[recap] = current_announcement_value
            continue

        print(f"\n[{idx+1}/{len(df)}] HISTORY_RECAP for {company_name}, old parent = {old_parent}, new parent = {new_parent}:\n{recap}\n")
        user_input = input("Enter acquisition exact date (MM/DD/YYYY) or press Enter to skip: ").strip()

        if user_input:
            df.at[idx, "acquisition_exact_date"] = user_input
            acquisition_cache[recap] = user_input
        
        user_input = input("Enter acquisition ANNOUNCEMENT exact date (MM/DD/YYYY) or press Enter to skip: ").strip()

        if user_input:
            df.at[idx, "acquisition_announcement_exact_date"] = user_input
            acquisition_announcement_cache[recap] = user_input

    # Save final state after loop
    df.to_excel(OUTPUT_FILE, index=False)
    print(f"\nFinal data saved to {OUTPUT_FILE}")

Loaded 5 cached acquisition dates from 'VT_PHMSA_parentchanged_with_dates.xlsx'
'VT_PHMSA_parentchanged_with_dates.xlsx' found but missing acquisition_announcement_exact_date column; starting with empty cache.

Press ESC at any time to stop and save progress.


[1/24852] HISTORY_RECAP for DOMINION TRANSMISSION INC, old parent = dominion energy, new parent = berkshire hathaway:
Berkshire Hathaway acquired Dominion Energy Transmission, Inc. in November 2020


[513/24852] HISTORY_RECAP for ROCKIES EXPRESS PIPELINE LLC, old parent = tallgrass energy, new parent = blackstone:
Blackstone acquired Tallgrass in 2019


[856/24852] HISTORY_RECAP for AERA ENERGY LLC, old parent = shell plc, new parent = california resources:
Royal Dutch Shell changed its name to Shell PLC in 2022. Shell sold Aera to California Resources in 2024


[857/24852] HISTORY_RECAP for CALUMET SUPERIOR LLC, old parent = calumet specialty products, new parent = cenovus energy:
In 2017 Calumet sold the operation to a subsidi